In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import concatenate_datasets, load_dataset

# Load the pretrained LLaMA 2 model and tokenizer
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load new task dataset
new_task_dataset = load_dataset("ag_news", split="train[:10%]")  # News classification task

# Tokenize the new dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_new_task = new_task_dataset.map(tokenize_function, batched=True)

# Create a replay buffer with examples from the original task (e.g., IMDb dataset)
old_task_dataset = load_dataset("imdb", split="train[:1%]")  # Small subset of original task
tokenized_old_task = old_task_dataset.map(tokenize_function, batched=True)

# Combine the new task data with the replay buffer
combined_dataset = concatenate_datasets([tokenized_new_task, tokenized_old_task])


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./llama2-continual-learning",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
)

# Define a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save the continually learned model
trainer.save_model("./llama2-continual-learning")
